# A lőcsei szászok alkalmi nyomtatványai

Első lépésként a pandas könyvtár segítségével beolvasom az elkészült Excel fájlokat. 

In [147]:
import pandas
import networkx as nx
from pyvis import network
import matplotlib.pyplot as plt
import math
import textwrap

records = pandas.read_excel('kiadvanyok.xlsx', sheet_name='lőcsei vonatkozás')
records.columns = records.columns.str.strip()
records.set_index('RMNy',  inplace=True)
records.head()

,Unnamed: 0,év,típusa,esemény,a nyomtatvány rövid leírása,a nyomtatvány címzettjei,szerzők és a nyomtatványban említett kapcsolódó nevek
RMNy,,,,,,,
1076,4,1614,Gyászversek,halál,Thurzó Kristóf főispán felett mondott gyászversek,Thurzó Kristóf,"Holtzmann, Stephanus"
1230,18,1620,Oktató versek,beiktatás,A versek a lőcsei városi magistrátus és az ala...,"Cramer, Kaspar, id.","Musculus, Wolfgang"
1231,19,1620,Gyászbeszéd,halál,Stephanus Holtzmann (Xylander) szepesváraljai ...,"Holtzmann, Stephanus","Zabeler, Peter; Horlerus, Stanislaus"
1254,20,1621,Lakodalmi ének,esküvő,Daniel Cramer és Christina Wolf 1621. szeptemb...,"Cramer, Daniel; Wolf, Christina","Musculus, Wolfgang; Wolf, Johann"
1255,21,1621,Oktató versek,általános üdvözlés,Tizenkét vers az elöljáróknak szükséges tizenk...,NaN,"Musculus, Wolfgang"


In [38]:
citizens = pandas.read_excel('polgarok_final.xlsx')
citizens

,id,név,születési hely,névváltozat,foglalkozás,életrajzi hivatkozás,rövid leírás,RMNy (alkalmi)
0,NaN,"Alauda, Barthalomeus (?-?)",NaN,"Alauda, Bartholomaeus",lőcsei szlovák evangélikus lelkipásztor,NaN,"1635-ben elvette Susanna Lapschert, Andreas He...",1625; 1681; 2014
1,NaN,"Alauda, Josephus (1610 k.-1664)",Lőcse (Szepes),NaN,a lőcsei evangélikus gimnázium tanára,lectio/rmsz1/56,Részt vett David Frölich és Judith Bertram esk...,1622; 1625; 1681; 1731; 1837; 1936; 2177; 2232
2,NaN,Alsópoprádvölgyi lelkipásztorok fraternitasa,NaN,Fratemitas pastorum ecclesiarum ad inferiorem ...,NaN,NaN,Johannes Serpilius (2.) az alsópoprádvölgyi le...,2185
3,NaN,"Apelles, Balthasar (?-?)",Szepes m.,"Apell; Appeles, Balthasar; B.A.",lőcsei kántor,lectio/rmsz1/126,1635-ben részt vett Barthalomeus Alauda és Sus...,1625; 1652; 1681; 1731; 1837
4,NaN,"Arelt, Gregorius (?-?)",Szombathely (Szepes),"Lerch, Josephus",NaN,lectio/rmsz1/138,Máriássy János és Palugyai Zsófia menyegzőjén ...,1620
...,...,...,...,...,...,...,...,...
202,NaN,"Wolf, Johann (?-?)",NaN,NaN,"nyomdász, több kutatást igényel",NaN,Részt vett Daniel Cramer és Christina Wolf esk...,1254
203,NaN,"Zabeler, Hiob (?-?)",NaN,"M. I. Z.; Z. I.; Zabler, Hiob",NaN,NaN,1648-ban verset írt Christoph Schlegel és Rosi...,2232
204,NaN,"Zabeler, Peter id. (1578-1645)",Grossenhain (Szászország),"Zabalerus, Petrus; Zabler Péter",evangélikus első lelkész és az észak-magyarors...,lectio/rmsz1/6367,"Wittenbergi magiszteri címmel jött Kassára, ma...",1231; 1443; 1681; 1830; 1832; 1837; 1934; 1936...
205,NaN,"Zabeler, Peter ifj. (?-?)",NaN,NaN,Matthaeus Saxo tanítványa Lőcsén.,NaN,Említésre kerül Matthaeus Saxo 1629-es logikat...,1231; 1443; 1830


Az alábbiakban a beolvasott nyomtatványok "címzettek" és "szerzők" oszlopait átolvasva, a nevekhez a polgár-adatbázisból hozzárendeljük a megfelelő polgárt, és egy relációs táblát készítünk a kettőből:

In [121]:
def get_from_citizen_list(name, record):
    index_name = citizens['név'].str.contains(name, na=False)
    index_alt_name = citizens['névváltozat'].str.contains(name, na=False)
    
    index_list = citizens[index_name]
    index_alt_list = citizens[index_alt_name]
    
    filtered_by_record = index_list['RMNy (alkalmi)'].astype(str).str.contains(str(record))
    filtered_by_record_alt = index_alt_list['RMNy (alkalmi)'].astype(str).str.contains(str(record))
    if not filtered_by_record.empty:
        return filtered_by_record.index[0]
    if not filtered_by_record_alt.empty:
        return filtered_by_record_alt.index[0]
    return math.nan


relations = []

for index, record in records.iterrows():
    recipient_names = record['a nyomtatvány címzettjei']
    sender_names = record['szerzők és a nyomtatványban említett kapcsolódó nevek']
    
    if isinstance(recipient_names, str) and isinstance(sender_names, str):
        recipients = recipient_names.split(';') 
        senders = sender_names.split(';')
        for recipient in recipients:
            target_id = get_from_citizen_list(recipient, index)
            
            for sender in senders:
                sender_id = get_from_citizen_list(sender, index)
                relation = {}
                relation['record_id'] = index
                relation['sender_name'] = sender
                if not math.isnan(sender_id):
                    relation['sender_id'] = int(sender_id)
                else:
                    relation['sender_id'] = -1
                
                relation['target_name'] = recipient
                if not math.isnan(target_id):
                    relation['target_id'] = int(target_id)
                else:
                    relation['target_id'] = -1
                
                relations.append(relation)
            

/tmp/ipykernel_2712/2385070799.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index_name = citizens['név'].str.contains(name, na=False)
/tmp/ipykernel_2712/2385070799.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  index_alt_name = citizens['névváltozat'].str.contains(name, na=False)


In [132]:
relations_df = pandas.DataFrame.from_records(relations)

relations_df['record_id']= relations_df['record_id'].astype(int)
relations_df['sender_name']= relations_df['sender_name'].astype(str)
relations_df['sender_id']= relations_df['sender_id'].astype(int)
relations_df['target_name']= relations_df['target_name'].astype(str)
relations_df['target_id']= relations_df['target_id'].astype(int)
                                             
relations_df.head()


,record_id,sender_name,sender_id,target_name,target_id
0,1076,"Holtzmann, Stephanus",-1,Thurzó Kristóf,185
1,1230,"Musculus, Wolfgang",125,"Cramer, Kaspar, id.",-1
2,1231,"Zabeler, Peter",204,"Holtzmann, Stephanus",-1
3,1231,"Horlerus, Stanislaus",-1,"Holtzmann, Stephanus",-1
4,1254,"Musculus, Wolfgang",125,"Cramer, Daniel",19


Az alábbiakban a relációs táblából a networkx könyvtár segítségével relációs gráfot készítek.

In [133]:
G = nx.from_pandas_edgelist(relations_df, source='sender_name', target='target_name', edge_attr=True, create_using=nx.MultiDiGraph())

In [151]:
net = network.Network(notebook=True, directed=True)
net.from_nx(G)


# Minden személyhez hozzáadjuk a rövid leírást, amit 
for edge in net.edges:
    to_node = net.get_node(edge['from'])
    from_node = net.get_node(edge['to'])
    to_node['title'] = "Nem található leírás."
    from_node['title'] = "Nem található leírás."
    
    if edge['sender_id'] != -1:
        string = citizens.iloc[edge['sender_id']]['rövid leírás']
        if isinstance(string, str):
            from_node['title'] = textwrap.fill(string, 40)
    if edge['target_id'] != -1:
        string = citizens.iloc[edge['target_id']]['rövid leírás']
        if isinstance(string, str):
            to_node['title'] = textwrap.fill(string, 40)
    
        
        
net.toggle_physics(True)
net.show('assets/relationship.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
